<center>
<img src="https://habrastorage.org/files/fd4/502/43d/fd450243dd604b81b9713213a247aa20.jpg" />
</center> 
     

## <center> Kaggle inclass competition from [mlcourse.ai](https://mlcourse.ai/)
    
# <center> [**Catch me if you can**](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2)

### <center> Session: Fall 2019

#### <div style="text-align: right"> Author: [Vladimir Kulyashov](https://github.com/koolvn)


<div style="text-align: right"> creation date: 15 October 2019 </div>

**Prerequisitions proved by EDA:**
1. Alice lives in France
2. Data was collected in the university at working hours 
3. Alice used PC mostly for watching videos and social networks
4. Alice doesn't use GMail or Google+ and Bing


**Goal:**
Beat the A3 strong baseline (0.95965) baseline with as less features, as possible

In [1]:
# Import libraries and set desired options
import os
import pickle
import numpy as np
import pandas as pd
from scipy.sparse import hstack
# !pip install eli5
import eli5
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression, SGDClassifier
from matplotlib import pyplot as plt
import seaborn as sns
from IPython.display import display_html

In [2]:
PATH_TO_DATA = '../data/alice/'
filename = f'submission_1.csv'
pred_path = './predictions/alice/'
SEED = 17
time_split = TimeSeriesSplit(n_splits=10)
logit = LogisticRegression(C=1, random_state=SEED, solver='liblinear')
sgdc= SGDClassifier(random_state=SEED)
BEST_LOGIT_C = 2.5118864315095824
BEST_LOGIT_TOL = 0.004

In [3]:
def prepare_sparse_features(path_to_train, path_to_test, path_to_site_dict,
                           vectorizer_params):
    """ Prepares sparsed X_train, X_test, y_train, vectorizer, train_times, test_times,
        train_sites, test_sites, top_alice_sites
        
        from input CSV files, pickle file and vectorizer_params dictionary.
    
        return:: X_train, X_test, y_train, vectorizer, train_times, test_times, train_sites, test_sites, top_alice_sites """
    
    times = ['time%s' % i for i in range(1, 11)]
    train_df = pd.read_csv(path_to_train,
                       index_col='session_id', parse_dates=times)
    test_df = pd.read_csv(path_to_test,
                      index_col='session_id', parse_dates=times)

    # Sort the data by time
    train_df = train_df.sort_values(by='time1')
    
    # read site -> id mapping provided by competition organizers 
    with open(path_to_site_dict, 'rb') as f:
        site2id = pickle.load(f)
    # create an inverse id _> site mapping
    id2site = {v:k for (k, v) in site2id.items()}
    # we treat site with id 0 as "unknown"
    id2site[0] = 'unknown'
    
    # Transform data into format which can be fed into TfidfVectorizer
    # This time we prefer to represent sessions with site names, not site ids. 
    # It's less efficient but thus it'll be more convenient to interpret model weights.
    sites = ['site%s' % i for i in range(1, 11)]
    train_sessions = train_df[sites].fillna(0).astype('int').apply(lambda row: 
                                                     ' '.join([id2site[i] for i in row]), axis=1).tolist()
    test_sessions = test_df[sites].fillna(0).astype('int').apply(lambda row: 
                                                     ' '.join([id2site[i] for i in row]), axis=1).tolist()
    
    sites_dict = pd.DataFrame(list(site2id.keys()),
                              index=list(site2id.values()),
                              columns=['site'])
    
    top_alice_sites = pd.Series(train_df[train_df['target'] == 1][sites].fillna(0).astype('int').values.flatten()
                               ).value_counts().sort_values(ascending=False).head(5)
    # we'll tell TfidfVectorizer that we'd like to split data by whitespaces only 
    # so that it doesn't split by dots (we wouldn't like to have 'mail.google.com' 
    # to be split into 'mail', 'google' and 'com')
    vectorizer = TfidfVectorizer(**vectorizer_params)
    X_train = vectorizer.fit_transform(train_sessions)
    X_test = vectorizer.transform(test_sessions)
    y_train = train_df['target'].astype('int').values
    
    # we'll need site visit times for further feature engineering
    train_times, test_times = train_df[times], test_df[times]
    
    # sites_df
    train_sites, test_sites = train_df[sites].fillna(0).astype('int'), test_df[sites].fillna(0).astype('int')
    
    full_df = pd.concat([train_df.drop('target', axis=1), test_df])
    
    return X_train, X_test, y_train, vectorizer, train_times, test_times, train_sites, test_sites, top_alice_sites

In [4]:
%%time
X_train_sites, X_test_sites, y_train, vectorizer, train_times, test_times, train_sites, test_sites, top_alice_sites = prepare_sparse_features(
    path_to_train=os.path.join(PATH_TO_DATA, 'train_sessions.csv'),
    path_to_test=os.path.join(PATH_TO_DATA, 'test_sessions.csv'),
    path_to_site_dict=os.path.join(PATH_TO_DATA, 'site_dic.pkl'),
    vectorizer_params={'ngram_range': (1, 5), 
                       'max_features': 50000,
                       'tokenizer': lambda s: s.split()}
)


Wall time: 32 s


In [5]:
# A helper function for writing predictions to a file and write list of features of that predictions
def write_to_submission_file(predicted_labels, out_file, new_feature_names=None, best_params=None, best_score=None,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)
    
    text_file = open(out_file+'.txt', "w")
    text_file.write(f'Features:\n{str(new_feature_names)}\nParams: {best_params}\nBest Score: {best_score}')
    text_file.close()

# I'm lazy, so here is a function that names files for you    
def get_next_filename(filename='submission_1.csv', path=pred_path, file_exists=False, i=1):
    if filename in os.listdir(path):
        file_exists = True
        while file_exists:
            i += 1
            next_ = list(filename.split('.')[0].split('_')[0])
            next_.append('_')
            next_.append(str(i))
            next_ = ''.join(next_) + '.csv'
            next_, file_exists = get_next_filename(next_, path, False, i)
            
        return next_, file_exists  
    else:
        file_exist = False
        return filename, file_exists
    
def train_and_predict(model, X_train, y_train, X_test, site_feature_names=vectorizer.get_feature_names(), 
                      new_feature_names=None, cv=time_split, scoring='roc_auc',
                      top_n_features_to_show=30, submission_file_name='submission.csv', best_params=None):
    
    
    cv_scores = cross_val_score(model, X_train, y_train, cv=cv, 
                            scoring=scoring, n_jobs=4)
    print('CV scores', cv_scores)
    print('\nCV mean: {}, CV std: {}'.format(cv_scores.mean(), cv_scores.std()))
    model.fit(X_train, y_train)
    
    if new_feature_names:
        all_feature_names = site_feature_names + new_feature_names 
    else: 
        all_feature_names = site_feature_names
    
    display_html(eli5.show_weights(estimator=model, 
                  feature_names=all_feature_names, top=top_n_features_to_show))
    
    if new_feature_names:
        print('New feature weights:')
    
        print(pd.DataFrame({'feature': new_feature_names, 
                        'coef': model.coef_.flatten()[-len(new_feature_names):]}).sort_values(by='coef'))
    
    test_pred = model.predict_proba(X_test)[:, 1]
    write_to_submission_file(test_pred, submission_file_name, new_feature_names, best_params,
                             best_score=f'\nCV max: {cv_scores.max()} CV mean: {cv_scores.mean()}, CV std: {cv_scores.std()}') 
    
    return cv_scores

### Base score with no features added

In [6]:
%%time
cv_scores_base = train_and_predict(model=logit,
                               X_train=X_train_sites,
                               y_train=y_train,
                               X_test=X_test_sites,
                               site_feature_names=vectorizer.get_feature_names(),
                               top_n_features_to_show=50,
                               cv=time_split, submission_file_name=pred_path+'base_'+filename)

CV scores [0.83124023 0.65993466 0.85673565 0.92824237 0.84779639 0.88954524
 0.88829128 0.8771044  0.92023038 0.92624125]

CV mean: 0.8625361862232206, CV std: 0.0745567081201155


Weight?,Feature
+5.880,youwatch.org
+5.380,cid-ed6c3e6a5c6608a4.users.storage.live.com
+5.222,fr.glee.wikia.com
+5.114,vk.com
+4.875,www.info-jeunes.net
+4.499,www.banque-chalus.fr
+4.220,www.express.co.uk
+4.147,www.audienceinsights.net
+4.089,www.melty.fr
+4.003,glee.hypnoweb.net


Wall time: 10.5 s


-----------------------------------------

### Adding features

### Feature engineering fields

In [7]:
with open(PATH_TO_DATA + 'site_dic.pkl', "rb") as input_file:
        site_dict = pickle.load(input_file)
        
facebook_ids = []
youtube_ids = []
google_ids = []
france_ids = []
sowftware_ids = []
wiki_ids = []
unknown_ids = []

for key in list(site_dict.keys()):
    if 'facebook' in key:
        facebook_ids.append(site_dict[key])
    if 'youtube' in key or 'youwatch' in key:
        youtube_ids.append(site_dict[key])
    if 'google' in key:
        google_ids.append(site_dict[key])
    if '.fr' in key:
        france_ids.append(site_dict[key])
    if 'browser-update.org' in key:
        sowftware_ids.append(site_dict[key])
    if 'wiki' in key:
        wiki_ids.append(site_dict[key])
    if 'unknown' in key:
        unknown_ids.append(site_dict[key])
            
sites_dict = pd.DataFrame(list(site_dict.keys()),
                          index=list(site_dict.values()),
                          columns=['site'])

scaler = StandardScaler()
sites_dict.loc[top_alice_sites.index]

,site
77,i1.ytimg.com
80,s.youtube.com
76,www.youtube.com
29,www.facebook.com
21,www.google.fr


In [8]:
times = ['time%s' % i for i in range(1, 11)]
sites = ['site%s' % i for i in range(1, 11)]

train_df = pd.read_csv(PATH_TO_DATA + 'train_sessions.csv',
                       index_col='session_id', parse_dates=times)

top_alice_sites = pd.Series(train_df[train_df['target'] == 1][sites].fillna(0).astype('int').values.flatten()
                               ).value_counts().sort_values(ascending=False).head(3)

In [9]:
train_df[train_df['target'] == 1][sites].fillna(0).astype('int').values.flatten()

array([5397, 5395,   22, ...,  879,   80,  879])

In [10]:
(train_times['time1'].dt.month == 5).astype('int').value_counts()

0    250332
1      3229
Name: time1, dtype: int64

In [11]:
((train_times['time1'].dt.month == 10) | (train_times['time1'].dt.month == 5)).astype('int').value_counts()

0    247544
1      6017
Name: time1, dtype: int64

In [12]:
len(wiki_ids)

238

The reason of choosing those features is simple - trying to "overfit" to Alice.

In [13]:
def add_features(times, sites, X_sparse, top_alice_sites):
    
    scaler = StandardScaler()
#     scaler = MinMaxScaler()
    
    with open(PATH_TO_DATA + 'site_dic.pkl', "rb") as input_file:
        site_dict = pickle.load(input_file)
        
        
    sites_dict = pd.DataFrame(list(site_dict.keys()),
                              index=list(site_dict.values()),
                              columns=['site'])
        
    # time features
    hour = times['time1'].dt.hour
    morning = ((hour >= 7) & (hour <= 11)).astype('int').values.reshape(-1, 1)
    day = ((hour >= 12) & (hour <= 18)).astype('int').values.reshape(-1, 1)
    evening = ((hour >= 19) & (hour <= 23)).astype('int').values.reshape(-1, 1)
    night = ((hour >= 0) & (hour <=6)).astype('int').values.reshape(-1, 1)
    
    durations = (times.max(axis=1) - times.min(axis=1)).astype('timedelta64[ms]').astype('int').values.reshape(-1, 1)
    durations = scaler.fit_transform(durations)
    
    week = times['time1'].dt.week.values.reshape(-1, 1)
    week = scaler.fit_transform(week)

    winter = times['time1'].dt.month.isin([12, 1, 2]).astype('int').values.reshape(-1, 1)
    spring = times['time1'].dt.month.isin([3, 4, 5]).astype('int').values.reshape(-1, 1)
    summer = times['time1'].dt.month.isin([6, 7, 8]).astype('int').values.reshape(-1, 1)
    autumn = times['time1'].dt.month.isin([9, 10, 11]).astype('int').values.reshape(-1, 1)
    
    day_of_week = times['time1'].dt.weekday.astype('int').values.reshape(-1, 1)
    month = times['time1'].dt.month.astype('int').values.reshape(-1, 1)
    year_month = times['time1'].apply(lambda ts: 100 * ts.year + ts.month).astype('int').values.reshape(-1, 1)
    year_month = scaler.fit_transform(year_month)
    
    sunday = (times['time1'].dt.weekday == 6).astype('int').values.reshape(-1, 1)
    
    may = (times['time1'].dt.month == 5).astype('int').values.reshape(-1, 1)
    october = (times['time1'].dt.month == 10).astype('int').values.reshape(-1, 1)
    
    # site features
    facebook_ids = []
    youtube_ids = []
    google_ids = []
    france_ids = []
    vk_ids = []
    msft_ids = []
    bing_ids = []
    unknown_ids = []
    search_ids = []

    for key in list(site_dict.keys()):
        if 'facebook' in key:
            facebook_ids.append(site_dict[key])
        if 'youtube' in key  in key or 'video' in key or 'youwatch' in key:# or 'ytimg.com' in key:
            youtube_ids.append(site_dict[key])
        if 'mail.google.com' in key or 'plus.google.com' in key: # or 'wwww.bing.com' in key:
            google_ids.append(site_dict[key])
        if '.fr' in key or 'fr.' in key:
            france_ids.append(site_dict[key])
        if 'vk.com' in key or 'vk.ru' in key:
            vk_ids.append(site_dict[key])
        if 'storage.live' in key or '.live.com' in key or 'fr.msn.com' in key:
            msft_ids.append(site_dict[key])
        if 'www.bing.com' in key:
            bing_ids.append(site_dict[key])
        if 'www.google.fr' in key:
            search_ids.append(site_dict[key])

    top_alice_ids = []

    for key in top_alice_sites.index:
        top_alice_ids.append(key)
    
    first_3 = sites[['site1', 'site2', 'site3']]

    in_alice_top = first_3.isin(top_alice_sites).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    
    
    start_google = first_3.isin(google_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    has_google = sites.isin(google_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    
    start_youtube = sites['site1'].isin(youtube_ids).astype('int').values.reshape(-1, 1)
    has_youtube = sites.isin(youtube_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    first_3_has_youtube = first_3.isin(youtube_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    
    start_facebook = sites['site1'].isin(facebook_ids).astype('int').values.reshape(-1, 1)
    has_facebook = sites.isin(facebook_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    first_3_has_facebook = first_3.isin(facebook_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    
    start_vk = sites['site1'].isin(vk_ids).astype('int').values.reshape(-1, 1)
    has_vk = sites.isin(vk_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    first_3_has_vk = first_3.isin(vk_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    
    fr_domains = sites.isin(france_ids).astype('int').sum(axis=1).values.reshape(-1, 1)
    fr_domains = scaler.fit_transform(fr_domains)
#     fr_domains = sites.isin(france_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    
    msft_usage = sites.isin(msft_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    
    start_bing = first_3.isin(bing_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    
    search = first_3.isin(search_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    
    # stacking matrix
    objects_to_hstack = [X_sparse, morning, day, evening, night, durations,
                         day_of_week,
                         year_month,
                         sunday,
                         start_google,
                         start_youtube,
                         fr_domains,
                         start_vk,
                         msft_usage,
                         summer,
#                          week,
                         has_google,
                         has_youtube,
                         in_alice_top,
                         has_vk,
                         has_facebook,
                         may,
                         october,
                         start_bing,
                         first_3_has_vk,
                         first_3_has_youtube,
                         first_3_has_facebook,
#                          search,
                         
                        ]
    
    feature_names = ['morning', 'day','evening', 'night', 'durations',
                     'day_of_week',
                     'year_month',
                     'sunday',
                     'start_google',
                     'start_youtube',
                     'fr_domains',
                     'start_vk',
                     'msft_usage', 
                     'summer',
#                      'week',
                     'has_google',
                     'has_youtube',
                     'in_alice_top',
                     'has_vk',
                     'has_facebook',
                     'may',
                     'october',
                     'start_bing',
                     'first_3_has_vk',
                     'first_3_has_youtube',
                     'first_3_has_facebook',
#                      'search',
                    ]
        
    X = hstack(objects_to_hstack)
    return X, feature_names

### Training model with new features

In [14]:
%%time
X_train, new_feat_names = add_features(train_times, train_sites, X_train_sites, top_alice_sites)
X_test, _ = add_features(test_times, test_sites, X_test_sites, top_alice_sites)

Wall time: 4.02 s


In [15]:
new_feat_names

['morning',
 'day',
 'evening',
 'night',
 'durations',
 'day_of_week',
 'year_month',
 'sunday',
 'start_google',
 'start_youtube',
 'fr_domains',
 'start_vk',
 'msft_usage',
 'summer',
 'has_google',
 'has_youtube',
 'in_alice_top',
 'has_vk',
 'has_facebook',
 'may',
 'october',
 'start_bing',
 'first_3_has_vk',
 'first_3_has_youtube',
 'first_3_has_facebook']

In [16]:
%%time
cv_scores_engineered = train_and_predict(model=logit, X_train=X_train, y_train=y_train,
                                         X_test=X_test, 
                                         site_feature_names=vectorizer.get_feature_names(),
                                         new_feature_names=new_feat_names,
                                         cv=time_split,
                                         submission_file_name=pred_path + 'engineered_' + filename)

CV scores [0.84162377 0.8568826  0.94186017 0.96896791 0.91585752 0.95494195
 0.92837104 0.9453201  0.96474345 0.96899149]

CV mean: 0.928755999474663, CV std: 0.04311277293396999


Weight?,Feature
+5.037,cid-ed6c3e6a5c6608a4.users.storage.live.com
+4.977,www.express.co.uk
+4.805,www.info-jeunes.net
+4.398,www.audienceinsights.net
+4.364,fr.glee.wikia.com
+4.080,youwatch.org
+4.037,www.melty.fr
+4.025,api.bing.com
+3.878,www.banque-chalus.fr
+3.788,dub119.mail.live.com


New feature weights:
                 feature      coef
14            has_google -3.796569
0                morning -3.109479
2                evening -2.614380
13                summer -2.540712
7                 sunday -1.706469
20               october -1.700759
8           start_google -0.914892
19                   may -0.892203
21            start_bing -0.794901
6             year_month -0.483992
5            day_of_week -0.355873
4              durations -0.229672
12            msft_usage -0.174409
11              start_vk -0.152493
16          in_alice_top  0.000000
3                  night  0.000000
24  first_3_has_facebook  0.008851
9          start_youtube  0.095077
10            fr_domains  0.168633
23   first_3_has_youtube  0.374082
18          has_facebook  0.445466
22        first_3_has_vk  0.494589
15           has_youtube  0.542901
1                    day  0.622331
17                has_vk  1.660874
Wall time: 25.9 s


In [17]:
cv_scores_base < cv_scores_engineered

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

Leaderboard score = 0.96215

### Parameter tuning

In [18]:
%%time
logit = LogisticRegression(random_state=SEED, solver='liblinear')

# params = {'C': np.logspace(0, 1.05, 10), 'tol': np.linspace(0.1, 0.0001, 10)}

params = {'C': [BEST_LOGIT_C, 2.2387211385683394, 2.9286445646252366],
          'tol': [BEST_LOGIT_TOL, 0.0556, 0.001]}


logit_grid_searcher = GridSearchCV(estimator=logit, param_grid=params,
                              scoring='roc_auc', n_jobs=4, cv=time_split, verbose=1)

logit_grid_searcher.fit(X_train, y_train)

print('Tuned score', logit_grid_searcher.best_score_)

Fitting 10 folds for each of 9 candidates, totalling 90 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   57.5s
[Parallel(n_jobs=4)]: Done  90 out of  90 | elapsed:  2.1min finished


Tuned score 0.9308356188664244
Wall time: 2min 16s


In [19]:
logit_grid_searcher.best_estimator_

LogisticRegression(C=2.2387211385683394, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=17, solver='liblinear', tol=0.004, verbose=0,
                   warm_start=False)

### Training the best model and writing the output files:
submission.csv + text file with features and best parameters

In [20]:
%%time
cv_scores_tuned = train_and_predict(model=logit_grid_searcher.best_estimator_, X_train=X_train, y_train=y_train,
                                         X_test=X_test, 
                                         site_feature_names=vectorizer.get_feature_names(),
                                         new_feature_names=new_feat_names,
                                         cv=time_split,
                                         submission_file_name=pred_path + get_next_filename(filename)[0], best_params=logit_grid_searcher.best_params_)

CV scores [0.85142668 0.84849918 0.94567793 0.97146672 0.92323371 0.95569686
 0.93333366 0.94419249 0.9666181  0.96821088]

CV mean: 0.9308356188664243, CV std: 0.043004723044531505


Weight?,Feature
+9.066,www.express.co.uk
+8.131,cid-ed6c3e6a5c6608a4.users.storage.live.com
+5.756,browser-update.org
+5.728,tru.am
+5.390,fr.glee.wikia.com
+5.341,www.info-jeunes.net
+5.244,api.bing.com
+5.010,www.banque-chalus.fr
+4.954,www.kelbillet.com
+4.838,www.audienceinsights.net


New feature weights:
                 feature      coef
14            has_google -4.367298
13                summer -3.166628
0                morning -3.111723
2                evening -3.109305
20               october -2.220347
7                 sunday -1.975547
19                   may -1.167675
8           start_google -0.973000
21            start_bing -0.626240
6             year_month -0.533039
5            day_of_week -0.357640
12            msft_usage -0.337007
4              durations -0.210149
11              start_vk -0.205840
24  first_3_has_facebook -0.018689
16          in_alice_top  0.000000
3                  night  0.000000
9          start_youtube  0.081534
10            fr_domains  0.149638
23   first_3_has_youtube  0.320660
18          has_facebook  0.396702
15           has_youtube  0.422965
22        first_3_has_vk  0.614814
1                    day  0.701931
17                has_vk  1.618162
Wall time: 26.3 s


In [21]:
cv_scores_engineered < cv_scores_tuned

array([ True, False,  True,  True,  True,  True,  True, False,  True,
       False])

In [22]:
# working hours feature + mean the latest 3 submissions + weekend feature + mail feature

Leaderboard score: **0.96247**

http://browser-update.org/ - looks like Alice has a 3rd party app installed.
www.express.co.uk - Alice reads news often

Let's find out.

In [23]:
def add_features(times, sites, X_sparse, top_alice_sites):
    
    scaler = StandardScaler()
#     scaler = MinMaxScaler()
    
    with open(PATH_TO_DATA + 'site_dic.pkl', "rb") as input_file:
        site_dict = pickle.load(input_file)
        
        
    sites_dict = pd.DataFrame(list(site_dict.keys()),
                              index=list(site_dict.values()),
                              columns=['site'])
        
    # time features
    hour = times['time1'].dt.hour
    morning = ((hour >= 7) & (hour <= 11)).astype('int').values.reshape(-1, 1)
    day = ((hour >= 12) & (hour <= 18)).astype('int').values.reshape(-1, 1)
    evening = ((hour >= 19) & (hour <= 23)).astype('int').values.reshape(-1, 1)
    night = ((hour >= 0) & (hour <=6)).astype('int').values.reshape(-1, 1)
    
    durations = (times.max(axis=1) - times.min(axis=1)).astype('timedelta64[ms]').astype('int').values.reshape(-1, 1)
    durations = scaler.fit_transform(durations)
    
    week = times['time1'].dt.week.values.reshape(-1, 1)
    week = scaler.fit_transform(week)

    winter = times['time1'].dt.month.isin([12, 1, 2]).astype('int').values.reshape(-1, 1)
    spring = times['time1'].dt.month.isin([3, 4, 5]).astype('int').values.reshape(-1, 1)
    summer = times['time1'].dt.month.isin([6, 7, 8]).astype('int').values.reshape(-1, 1)
    autumn = times['time1'].dt.month.isin([9, 10, 11]).astype('int').values.reshape(-1, 1)
    
    day_of_week = times['time1'].dt.weekday.astype('int').values.reshape(-1, 1)
    month = times['time1'].dt.month.astype('int').values.reshape(-1, 1)
    year_month = times['time1'].apply(lambda ts: 100 * ts.year + ts.month).astype('int').values.reshape(-1, 1)
    year_month = scaler.fit_transform(year_month)
    
    sunday = (times['time1'].dt.weekday == 6).astype('int').values.reshape(-1, 1)
    wednesday = (times['time1'].dt.weekday == 5).astype('int').values.reshape(-1, 1)

    may = (times['time1'].dt.month == 5).astype('int').values.reshape(-1, 1)
    october = (times['time1'].dt.month == 10).astype('int').values.reshape(-1, 1)
    
    
    # site features
    facebook_ids = []
    youtube_ids = []
    google_ids = []
    france_ids = []
    vk_ids = []
    msft_ids = []
    bing_ids = []
    software_ids = []
    alice_top_ids = []
    wiki_ids =[]
    

    for key in list(site_dict.keys()):
        if 'facebook' in key:
            facebook_ids.append(site_dict[key])
        if 'youtube' in key  in key or 'video' in key:# or 'ytimg.com' in key:
            youtube_ids.append(site_dict[key])
        if 'mail.google.com' in key or 'plus.google.com' in key: # or 'wwww.bing.com' in key:
            google_ids.append(site_dict[key])
        if '.fr' in key or 'fr.' in key:
            france_ids.append(site_dict[key])
        if 'vk.com' in key or 'vk.ru' in key:
            vk_ids.append(site_dict[key])
        if 'storage.live' in key or '.live.com' in key:
            msft_ids.append(site_dict[key])
        if 'www.bing.com' in key:
            bing_ids.append(site_dict[key])
        if 'news' in key or 'mail' in key or 'express' in key:
            software_ids.append(site_dict[key])
        if 'wiki' in key:
            wiki_ids.append(site_dict[key])
        
#         if 'www.google.fr' in key or 'www.youtube.com' in key or 's.youtube.com' in key or 'www.facebook.com' in key or 'i1.ytimg.com' in key:
#             alice_top_ids.append(site_dict[key])
            
        
    sites_dict.loc[top_alice_sites.index]
    top_alice_ids = []

    for key in top_alice_sites.index:
        top_alice_ids.append(key)

    in_alice_top = sites.isin(top_alice_sites).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    
    start_google = sites['site1'].isin(google_ids).astype('int').values.reshape(-1, 1)
    has_google = sites.isin(google_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    start_youtube = sites['site1'].isin(youtube_ids).astype('int').values.reshape(-1, 1)
    has_youtube = sites.isin(youtube_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    start_facebook = sites['site1'].isin(facebook_ids).astype('int').values.reshape(-1, 1)
    has_facebook = sites.isin(facebook_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    start_vk = sites['site1'].isin(vk_ids).astype('int').values.reshape(-1, 1)
    has_vk = sites.isin(vk_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    fr_domains = sites.isin(france_ids).astype('int').sum(axis=1).values.reshape(-1, 1)
    fr_domains = scaler.fit_transform(fr_domains)
#     fr_domains = sites.isin(france_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    msft_usage = sites.isin(msft_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    
#     start_bing = sites['site1'].isin(bing_ids).astype('int').values.reshape(-1, 1)
    has_bing = sites.isin(bing_ids).astype('int').astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    has_software = sites.isin(sowftware_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    has_alice_top = sites.isin(top_alice_ids).astype('int').sum(axis=1).values.reshape(-1, 1)
    has_alice_top = scaler.fit_transform(has_alice_top)
    wiki = sites.isin(wiki_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    
    # stacking matrix
    objects_to_hstack = [X_sparse, morning, day, evening, night,
#                          durations,
                         day_of_week,
                         year_month,
                         sunday,
                         start_google,
                         start_youtube,
                         fr_domains,
#                          start_vk,
                         msft_usage,
                         summer,
#                          week,
                         has_google,
                         has_youtube,
#                          in_alice_top,
                         has_vk,
                         has_facebook,
                         may,
                         october,
#                          wednesday,
#                          has_software,
#                          has_bing,
#                          has_alice_top,
                         wiki
                         
                        ]
    
    feature_names = ['morning', 'day','evening', 'night',
#                      'durations',
                     'day_of_week',
                     'year_month',
                     'sunday',
                     'start_google',
                     'start_youtube',
                     'fr_domains',
#                      'start_vk',
                     'msft_usage', 
                     'summer',
#                      'week',
                     'has_google',
                     'has_youtube',
#                      'in_alice_top',
                     'has_vk',
                     'has_facebook',
                     'may',
                     'october',
#                      'wednesday',
#                      'has_software',
#                      'has_bing',
#                      'has_alice_top',
                     'wiki'
                    ]
        
    X = hstack(objects_to_hstack)
    return X, feature_names

X_train, new_feat_names = add_features(train_times, train_sites, X_train_sites, top_alice_sites)
X_test, _ = add_features(test_times, test_sites, X_test_sites, top_alice_sites)

In [24]:
%%time
cv_scores_tuned_new = train_and_predict(model=logit_grid_searcher.best_estimator_, X_train=X_train, y_train=y_train,
                                         X_test=X_test, 
                                         site_feature_names=vectorizer.get_feature_names(),
                                         new_feature_names=new_feat_names,
                                         cv=time_split,
                                         submission_file_name=pred_path + get_next_filename(filename)[0],
                                         best_params=logit_grid_searcher.best_params_)

CV scores [0.85248157 0.85114113 0.94425832 0.96980266 0.92125048 0.9211636
 0.93296528 0.94333645 0.96670364 0.96856328]

CV mean: 0.9271666404187829, CV std: 0.04136504965829113


Weight?,Feature
+9.167,www.express.co.uk
+8.263,cid-ed6c3e6a5c6608a4.users.storage.live.com
+5.971,browser-update.org
+5.696,tru.am
+5.569,fr.glee.wikia.com
+5.333,www.info-jeunes.net
+5.156,youwatch.org
+5.108,api.bing.com
+4.987,www.banque-chalus.fr
+4.967,www.kelbillet.com


New feature weights:
          feature      coef
12     has_google -4.562985
11         summer -3.165073
0         morning -3.114391
2         evening -3.102007
17        october -2.230275
6          sunday -1.981057
16            may -1.162680
7    start_google -0.538727
5      year_month -0.532449
4     day_of_week -0.355489
10     msft_usage -0.327835
18           wiki -0.202160
3           night  0.000000
8   start_youtube  0.131581
9      fr_domains  0.147274
15   has_facebook  0.407330
13    has_youtube  0.501349
1             day  0.716032
14         has_vk  1.812194
Wall time: 20.9 s


In [25]:
cv_scores_tuned_new > cv_scores_tuned

array([ True,  True, False, False, False, False, False, False,  True,
        True])

1. with wednesday - 0.95832 on LB
1. with software - 0.95771 on LB
1. with has_bing - 0.96226 on LB

In [26]:
logit_grid_searcher.best_params_

{'C': 2.2387211385683394, 'tol': 0.004}